<a href="https://www.kaggle.com/code/dascient/yahoo-s-top-gainers-kalman-forecasting-model?scriptVersionId=216325137" target="_blank"><img align="left" alt="Kaggle" title="Open in Kaggle" src="https://kaggle.com/static/images/open-in-kaggle.svg"></a>

# Welcome to our Universe
## [@donutz.ai](www.donutz.ai/#)

- Information and market data collected from Yahoo - https://finance.yahoo.com/gainers/
- Please do not consider the recommendations below as valid. Trade at your own discretion. 

In [ ]:
# source
# https://github.com/twopirllc/pandas-ta/blob/main/examples/example.ipynb

# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

!pip install yfinance --upgrade --no-cache-dir
!pip install plotly
!pip install pykalman
!pip install pandas_ta

from IPython.display import clear_output
import plotly.graph_objects as go
import warnings,time,ast
from pykalman import KalmanFilter
import plotly.graph_objects as go
warnings.filterwarnings('ignore')
import matplotlib.pyplot as plt
from dateutil.tz import tzlocal
from rich import print, pretty
from datetime import datetime
from random import randint
from random import seed
import pandas_ta as ta
import yfinance as yf
import pandas as pd
import numpy as np
import lxml.html
import requests
import os

clear_output()

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
def calculate_symbol(df):
    
    df['close'] = df['Close']
    df['low'] = df['Low']
    df['high'] = df['High']
    df['open'] = df['Open']
    
    close = df['close'][len(df)-1]
    low = df['low'][len(df)-1]

    # Construct a Kalman filter
    kf = KalmanFilter(transition_matrices = [1],    # The value for At. It is a random walk so is set to 1.0
                      observation_matrices = [1],   # The value for Ht.
                      initial_state_mean = 0,       # Any initial value. It will converge to the true state value.
                      initial_state_covariance = 1, # Sigma value for the Qt in Equation (1) the Gaussian distribution
                      observation_covariance=1,     # Sigma value for the Rt in Equation (2) the Gaussian distribution
                      transition_covariance=.01)    # A small turbulence in the random walk parameter 1.0
    # Get the Kalman smoothing
    state_means, _ = kf.filter(df['close'].values)
    # Call it kf_mean
    df['kf_mean'] = np.array(state_means)
    kalman = df.kf_mean[len(df)-1]
    aboveKalman = low > kalman


    # exponential moving averages 
    ema_13 = df.ta.ema(13, append=True)[-1:].reset_index(drop=True)[0]
    ema_31 = df.ta.ema(31, append=True)[-1:].reset_index(drop=True)[0]
    ema_crossover = ema_13 > kalman


    # lower/upper 14-day bollinger bands for mean reversion
    bbl_14 = df.ta.bbands(length=14, append=True)[['BBL_14_2.0']].tail(1).values[0][0]
    bbu_14 = df.ta.bbands(length=14, append=True)[['BBU_14_2.0']].tail(1).values[0][0]
    bband_buy = close < bbl_14
    bband_sell = close > bbu_14


    # ichimoku 9 & 26-day forecasts 
    # https://technical-analysis-library-in-python.readthedocs.io/en/latest/ta.html#ta.trend.IchimokuIndicator
    isa_9 = df.ta.ichimoku()[1]['ISA_9'].tail(1).values[0] # help(ta.ichimoku)
    isb_26 = df.ta.ichimoku()[1]['ISB_26'].tail(1).values[0]


    # archer ma 
    # https://github.com/twopirllc/pandas-ta#general
    amat = (df.ta.amat()['AMATe_LR_8_21_2'].tail(1).values[0] == 1)


    # rsi
    rsi = df.ta.rsi()[len(df)-1]
    rsi_buy = rsi < 30
    rsi_sell = rsi > 70


    # choppy
    # https://github.com/twopirllc/pandas-ta#trend-18
    try: 
        chop = "{:.2f}".format(df.ta.chop()[len(df.ta.chop())-1]) 
    except RunTimeWarning:
        chop = 0


    # ~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~ #
    # signal
    #buy = (close < isa_9) & (close < isb_26) & amat & rsi_buy & bband_buy & aboveKalman
    
    # kalman is accounted for in the ema_crossover
    buy = amat & ema_crossover# & aboveKalman

    #sell = (close > isa_9) & (close > isb_26) & ~amat & rsi_sell & bband_sell & ~aboveKalman
    
    # kalman is accounted for in the ema_crossover
    sell = ~amat & ~ema_crossover# & ~aboveKalman
    
    return df, close, isa_9, isb_26, chop, rsi, amat, ema_crossover, buy, sell, aboveKalman

# plotter
def plot(df,symbol):
    fig = go.Figure(go.Candlestick(x=df.index,
                                   open=df['open'],
                                   high=df['high'],
                                   low=df['low'],
                                   close=df['close'],
                                   name=symbol,
                                  ))

    fig.add_trace(go.Scatter(x=df.index, 
                             y=df['kf_mean'], 
                             opacity=0.7, 
                             line=dict(color='purple', width=2), 
                             name='Kalman Filter'))

    fig.add_trace(go.Scatter(x=df.index, 
                             y=df['EMA_13'], 
                             opacity=0.7, 
                             line=dict(color='orange', width=2), 
                             name='EMA_13'))
    
    fig.add_trace(go.Scatter(x=df.index, 
                             y=df['EMA_31'], 
                             opacity=0.7, 
                             line=dict(color='lightblue', width=2), 
                             name='EMA_31 '))
    
    fig.update_layout(title=f'Ticker: {symbol}')
    fig.update_layout(xaxis_rangeslider_visible=False)
    return fig.show()

# [Fear & Greed Index](https://www.cnn.com/markets/fear-and-greed)

In [ ]:
!pip install fear-and-greed
clear_output()

import fear_and_greed
#type(fear_and_greed.get())

print('\nIndex: ',fear_and_greed.get()[0])
print('Range: ',fear_and_greed.get()[1])
print('Datetime: ',fear_and_greed.get()[2])

# Today's Yahoo Gainers 
## Including Recommendations, Candlesticks, Closing Price, Ichimoku 9 & 26 Day Spans, Choppiness, RSI, Exponential Moving Averages (EMA) & Kalman Forecast Predictors. 
### This notebook only plots entry & exit stocks out of the 100 from Yahoo's Gainers webpage.

In [6]:
from datetime import datetime
today = pd.Timestamp(datetime.now(), tz='EST').strftime("%Y-%m-%d")
print(f"Calculating indicators for today's date: {today}.")

Calculating indicators for today's date: 2024-12-23.

# Entry Points

In [86]:
import BeautifulSoup

# url = "https://finance.yahoo.com/gainers"
soup = BeautifulSoup(requests.get(url).text, 'html.parser')
assets = soup.find_all('a', attrs={"class":"Fw(600)"})

SyntaxError: invalid syntax (1542142144.py, line 1)

In [91]:
gainers = pd.read_html('https://opus.analytics.yahoo.com/tag/opus-frame.html?referrer=https%3A%2F%2Ffinance.yahoo.com%2Fmarkets%2Fstocks%2Fgainers%2F&amp;tbla_id=efa9c005-989d-4131-9a21-b4d36adb9f36-tucte142bee&amp;axids=gam%3Dy-vv8zzDFE2uIiFyYTqYsEuLmm9axZZzZr~A%26dv360%3DeS1TWHZzWnFKRTJ1R3BVRTlsU1JoYmN0bFZBYkhIOG55Qn5B%26ydsp%3Dy-vftJ9rRE2uILYIS6yHpo1EjCusPeUdxR~A%26tbla%3Dy-a_PikMBE2uK421wcXsiFqrr.vjdUlVLm~A&amp;gdpr=false&amp;gdpr_consent=&amp;gpp=DBABBg~BVoIgACY.QA&amp;gpp_sid=8&amp;us_privacy=1YNN')

ValueError: No tables found

In [64]:
# Plotly within a double nested for-loop. 
# This pulls from Yahoo's Gainers first page, then implements technical indicators to output forecasts, 
# RSI, choppiness, trend archers, & other conditionals to aid in decision making.

from plotly.offline import init_notebook_mode, iplot
init_notebook_mode(connected=True)
%matplotlib inline

# Pull Yahoo's Top Gainers of Today - first page.

url = 'https://finance.yahoo.com/gainers'
headers = {
  'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; rv:91.0) Gecko/20100101 Firefox/91.0'
}

ytext = requests.get(url,headers=headers).text
yroot = lxml.html.fromstring(ytext)
List = []
for x in yroot.xpath('//*[@id="fin-scr-res-table"]//a'):
    List.append({"top_movers":x.attrib['href'].split("/")[-2].split("?")[0]})
List = pd.DataFrame(List)
List



""


In [66]:
def printStocks(url):
    ytext = requests.get(url,headers=headers).text
    yroot = lxml.html.fromstring(ytext)
    for x in yroot.xpath('//*[@id="fin-scr-res-table"]//a'):
        print(x.attrib['href'].split("/")[-1].split("?")[0])

printStocks(url)



In [62]:
# Plotly within a double nested for-loop. 
# This pulls from Yahoo's Gainers first page, then implements technical indicators to output forecasts, 
# RSI, choppiness, trend archers, & other conditionals to aid in decision making.

from plotly.offline import init_notebook_mode, iplot
init_notebook_mode(connected=True)
%matplotlib inline

# Pull Yahoo's Top Gainers of Today - first page.

url = 'https://finance.yahoo.com/gainers'
headers = {
  'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; rv:91.0) Gecko/20100101 Firefox/91.0'
}

ytext = requests.get(url,headers=headers).text
yroot = lxml.html.fromstring(ytext)
List = []
for x in yroot.xpath('//*[@id="fin-scr-res-table"]//a'):
    List.append({"top_movers":x.attrib['href'].split("/")[-2].split("?")[0]})
List = pd.DataFrame(List)

# calculating plotter
print('\n Entry Points')
for i,v in [(v,yf.download(tickers=['ESPGY','ALYI'], period='3mo', interval='1d')) for i,v in List.close.items()]:
    try:
        #calculate_symbol(v)
        if (calculate_symbol(v)[8] == True) & (calculate_symbol(v)[4]<="40"):
            print(f'\n\nTicker: {i}\n\n "Entry Point"  AMAT &  (EMA_13 > Kalman) & (Choppiness <= 40):',calculate_symbol(v)[8])
            print(' "Exit Point"  ~AMAT & ~(EMA_13 > Kalman):',calculate_symbol(v)[9])
            print('\n\nClose:',calculate_symbol(v)[1])
            print('Ichimoku 9-Day Forecast:',calculate_symbol(v)[2])
            print('Ichimoku 26-Day Forecast:',calculate_symbol(v)[3])
            print('Choppiness (%):',calculate_symbol(v)[4])
            print('RSI:',calculate_symbol(v)[5])
            print('Archer MA Trending:',calculate_symbol(v)[6])
            print('EMA_13 > Kalman:',calculate_symbol(v)[7])
            print('Low > Kalman:',calculate_symbol(v)[10])
            plot(v.tail(90),i)
        else:pass
    except:pass

Entry Points

AttributeError: 'DataFrame' object has no attribute 'close'

In [47]:
yf.download(tickers=['ESPGY','ALYI'], period='1mo', interval='1d')

[*********************100%***********************]  2 of 2 completed


Price        Close           High            Low           Open         \
Ticker        ALYI  ESPGY    ALYI  ESPGY    ALYI  ESPGY    ALYI  ESPGY   
Date                                                                     
2024-11-25  0.0008  0.030  0.0008  0.050  0.0007  0.030  0.0007  0.040   
2024-11-26  0.0007  0.030  0.0008  0.040  0.0007  0.030  0.0007  0.030   
2024-11-27  0.0008  0.030  0.0008  0.040  0.0006  0.030  0.0006  0.030   
2024-11-29  0.0006  0.040  0.0008  0.040  0.0006  0.030  0.0006  0.040   
2024-12-02  0.0008  0.040  0.0009  0.040  0.0007  0.030  0.0007  0.040   
2024-12-03  0.0008  0.030  0.0009  0.040  0.0007  0.030  0.0009  0.030   
2024-12-04  0.0008  0.030  0.0008  0.040  0.0006  0.030  0.0007  0.030   
2024-12-05  0.0007  0.040  0.0008  0.040  0.0006  0.030  0.0006  0.040   
2024-12-06  0.0008  0.030  0.0010  0.040  0.0006  0.030  0.0006  0.040   
2024-12-09  0.0007  0.040  0.0009  0.040  0.0005  0.030  0.0008  0.040   
2024-12-10  0.0008  0.040  0.0009  0.040  0.0007  0.030  0.0007  0.040   
2024-12-11  0.0009  0.040  0.0010  0.040  0.0007  0.030  0.0007  0.040   
2024-12-12  0.0010  0.040  0.0011  0.050  0.0008  0.040  0.0008  0.040   
2024-12-13  0.0011  0.040  0.0011  0.040  0.0008  0.030  0.0008  0.040   
2024-12-16  0.0010  0.040  0.0011  0.040  0.0009  0.030  0.0010  0.030   
2024-12-17  0.0009  0.040  0.0013  0.040  0.0001  0.030  0.0011  0.040   
2024-12-18  0.0009  0.030  0.0011  0.040  0.0008  0.030  0.0009  0.040   
2024-12-19  0.0011  0.040  0.0013  0.040  0.0009  0.030  0.0009  0.030   
2024-12-20  0.0010  0.038  0.0011  0.038  0.0007  0.038  0.0007  0.038   
2024-12-23  0.0010    NaN  0.0010    NaN  0.0010    NaN  0.0010    NaN   

Price         Volume            
Ticker          ALYI     ESPGY  
Date                            
2024-11-25   1401020  772500.0  
2024-11-26   2132418  197200.0  
2024-11-27   4381323  193200.0  
2024-11-29   1146010   87500.0  
2024-12-02   3565183  193000.0  
2024-12-03   1322640  150800.0  
2024-12-04   5659234  162400.0  
2024-12-05   5643597   99500.0  
2024-12-06   6890193  126800.0  
2024-12-09   4223309  181200.0  
2024-12-10   5466789   71800.0  
2024-12-11   8104729  139900.0  
2024-12-12   7800577  585500.0  
2024-12-13    982488  137100.0  
2024-12-16   3974568  352000.0  
2024-12-17   6449847   58600.0  
2024-12-18  10729669  197000.0  
2024-12-19   5810848  183700.0  
2024-12-20   2883016   10400.0  
2024-12-23     60000       NaN

In [12]:
# Plotly within a double nested for-loop. 
# This pulls from Yahoo's Gainers first page, then implements technical indicators to output forecasts, 
# RSI, choppiness, trend archers, & other conditionals to aid in decision making.

from plotly.offline import init_notebook_mode, iplot
init_notebook_mode(connected=True)
%matplotlib inline

# Pull Yahoo's Top Gainers of Today - first page.

url = 'https://finance.yahoo.com/gainers?count=100&offset=0'
headers = {
  'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; rv:91.0) Gecko/20100101 Firefox/91.0'
}

ytext = requests.get(url,headers=headers).text
yroot = lxml.html.fromstring(ytext)
List = []
for x in yroot.xpath('//*[@id="fin-scr-res-table"]//a'):
    List.append({"top_movers":x.attrib['href'].split("/")[-2].split("?")[0]})
List = pd.DataFrame(List)

# calculating plotter
print('\n Entry Points')
for i,v in [(v,yf.download(tickers=['ESPGY','ALYI'], period='3mo', interval='1d')) for i,v in List.Volume.items()]:
    try:
        #calculate_symbol(v)
        if (calculate_symbol(v)[8] == True) & (calculate_symbol(v)[4]<="40"):
            print(f'\n\nTicker: {i}\n\n "Entry Point"  AMAT &  (EMA_13 > Kalman) & (Choppiness <= 40):',calculate_symbol(v)[8])
            print(' "Exit Point"  ~AMAT & ~(EMA_13 > Kalman):',calculate_symbol(v)[9])
            print('\n\nClose:',calculate_symbol(v)[1])
            print('Ichimoku 9-Day Forecast:',calculate_symbol(v)[2])
            print('Ichimoku 26-Day Forecast:',calculate_symbol(v)[3])
            print('Choppiness (%):',calculate_symbol(v)[4])
            print('RSI:',calculate_symbol(v)[5])
            print('Archer MA Trending:',calculate_symbol(v)[6])
            print('EMA_13 > Kalman:',calculate_symbol(v)[7])
            print('Low > Kalman:',calculate_symbol(v)[10])
            plot(v.tail(90),i)
        else:pass
    except:pass

Entry Points

AttributeError: 'DataFrame' object has no attribute 'Volume'

# Exit Points

In [ ]:
print('\n Exit Points')
for i,v in [(v,yf.download(tickers=v, period='3mo', interval='1d')) for i,v in List.top_movers.items()]:
    try:
        #calculate_symbol(v)
        if (calculate_symbol(v)[9] == True) & (calculate_symbol(v)[4] >= "50"):
            print(f'\n\nTicker: {i}\n\n "Entry Point"  AMAT &  (EMA_13 > Kalman) & (Choppiness >= 50):',calculate_symbol(v)[8])
            print(' "Exit Point"  ~AMAT & ~(EMA_13 > Kalman):',calculate_symbol(v)[9])
            print('\n\nClose:',calculate_symbol(v)[1])
            print('Ichimoku 9-Day Forecast:',calculate_symbol(v)[2])
            print('Ichimoku 26-Day Forecast:',calculate_symbol(v)[3])
            print('Choppiness (%):',calculate_symbol(v)[4])
            print('RSI:',calculate_symbol(v)[5])
            print('Archer MA Trending:',calculate_symbol(v)[6])
            print('EMA_13 > Kalman:',calculate_symbol(v)[7])
            print('Low > Kalman:',calculate_symbol(v)[10])
            plot(v.tail(90),i)
        else:pass
    except:pass

# Portfolio

In [ ]:
# calculating plotter
Portfolio = ['AAPL','AI','FCEL','NOC','PLTR','SBUX','TSLA']
for i,v in [(v,yf.download(tickers=v, period='3mo', interval='1d')) for v in Portfolio]: 
    print(f'\n\nTicker: {i}\n\n "Entry Point"  AMAT &  (EMA_13 > Kalman) & (Choppiness <= 40):',calculate_symbol(v)[8])
    print(' "Exit Point"  ~AMAT & ~(EMA_13 > Kalman):',calculate_symbol(v)[9])
    print('\n\nClose:',calculate_symbol(v)[1])
    print('Ichimoku 9-Day Forecast:',calculate_symbol(v)[2])
    print('Ichimoku 26-Day Forecast:',calculate_symbol(v)[3])
    print('Choppiness (%):',calculate_symbol(v)[4])
    print('RSI:',calculate_symbol(v)[5])
    print('Archer MA Trending:',calculate_symbol(v)[6])
    print('EMA_13 > Kalman:',calculate_symbol(v)[7])
    print('Low > Kalman:',calculate_symbol(v)[10])
    plot(v.tail(90),i)
else:pass

In [ ]:
# choppiness
print("Choppiness:")
help(ta.chop)

In [ ]:
# soon to incorporate this charting method: https://github.com/twopirllc/pandas-ta/blob/main/examples/example.ipynb

In [ ]:
# en fin

# Disclaimer

All investments involve risk, and the past performance of a security, industry, sector, market, financial product, trading strategy, or individual’s trading does not guarantee future results or returns. Investors are fully responsible for any investment decisions they make. Such decisions should be based solely on an evaluation of their financial circumstances, investment objectives, risk tolerance, and liquidity needs.

Any opinions, news, research, analyses, prices, or other information offered is provided as general market commentary, and does not constitute investment advice. We will not accept liability for any loss or damage, including without limitation any loss of profit, which may arise directly or indirectly from use of or reliance on such information.


# Supplementary

In [ ]:
ticker = 'NOC'
df = yf.download(tickers=ticker, period='3mo', interval='1d')
calculate_symbol(df)
plot(df,ticker)


In [ ]:
# Used for example Trend Return Long Trend Below
macd_ = ta.macd(df.Close)
macdh = macd_[macd_.columns[1]]
df.tail()

In [ ]:
# cumulative log return
clr_ma_length = 8
clrdf = df.ta.log_return(cumulative=True, append=True)
clrmadf = ta.ema(clrdf, length=clr_ma_length)
clrxdf = pd.DataFrame({f"{clrdf.name}": clrdf, f"{clrmadf.name}({clrdf.name})": clrmadf})
clrxdf.plot(figsize=(15,4), linewidth=1, title=f"{ticker}", grid=True)

In [ ]:
# macd
macddf = df.ta.macd(fast=8, slow=21, signal=9, min_periods=None, append=True)
macddf[[macddf.columns[0], macddf.columns[2]]].plot(figsize=(16, 2), linewidth=1.3)
macddf[macddf.columns[1]].plot.area(figsize=(15,4), stacked=False, color=["silver"], linewidth=1, title=f"{ticker}", grid=True).axhline(y=0, color="black", lw=1.1)